In [ ]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
from fast_sql_queries import *
import json
import csv
from sqlalchemy_schemadisplay import create_schema_graph
from sqlalchemy import MetaData
from monitor_script import profile

In [ ]:
def iter_song_data(filepath):
    """
    Iterator function for reading song JSON files in filepath with one row inside and yield data as
    dictionary
    
    Args:
        filepath (str) -> root file path contain all json file
    """
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            if 'checkpoint' not in f:
                all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))
    
    for file in all_files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        yield data
        
def song_data_extract(iter_file):
    """
    Function that proccess song file iterator and accumulate correct data from all the files into a list of tuples.
    
    Args:
        iter_file (iterator) -> Iterator for song files
    
    Returns:
        song_lst (list) --> List of tuples contain valid data songs
        artist_lst (list) --> List of tupels contai valid data for artist
    """
    song_columns = ["song_id", "title", "artist_id", "year", "duration"]
    artist_columns = ["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]
    song_lst = []
    artist_lst = []
    
    for file in iter_file:
        song_lst.append(tuple(file[k] for k in song_columns))
        artist_lst.append(tuple(file[k] for k in artist_columns))
        
    return song_lst,artist_lst


def log_file_extract(filepath):
    """
    Function to process log files and accumulate all file data into one pandas dataframe
    
    Args:
        filepath (str) -> root file path contain all json file
        
    Return:
        df (pandas.dataframe) --> Dataframe with all data from files 
    """
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            if 'checkpoint' not in f:
                all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))
    # collecting all files into dataframe
    df = pd.read_json(all_files[0], lines = True)
    for file in all_files[1:]:
        df = df.append(pd.read_json(file, lines = True))
    return df[df['page'] =='NextSong'] # filter data having page = NextSong
        
        
def log_data_extract(cur,df):
    """
    
    """
    def extract_song_artists_id(cur,row):
        cur.execute(song_select, (row['song'], row['artist'], row['length']))
        results = cur.fetchone()

        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None
        
        row['song_id'] = songid
        row['artist_id'] = artistid
        return row
    
    time_columns = ["ts", "hour", "day", "week", "month", "year", "weekday"]
    user_columns = ["userId", "firstName", "lastName", "gender", "level"]
    songplay_columns = ['ts', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent']
    
    #songplay data
    df['song_id'] = 0
    df['artist_id'] = 0
    df.apply(lambda x:extract_song_artists_id(cur,x),axis=1)
    df['ts'] = pd.to_datetime(df['ts'],unit='ms')       
    t = df['ts']
    time_data = [t.values,t.dt.hour,t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.dayofweek]
    time_df = pd.DataFrame({time_columns[i] : time_data[i] for i in range(len(time_data))})
    #print(df[user_columns].values[0])
    user_df = df[user_columns]
    #songplay data
    
    songplay_data = df[songplay_columns]
    return time_df,user_df,songplay_data

def insert_mogrify(cur,tuples, insert, values_statement, clause):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    """
    values = [cur.mogrify(values_statement, tup).decode('utf8') for tup in tuples]
    query  = insert + ",".join(values) + clause
    try:
        cur.execute(query)
        conn.commit()
        print("execute_mogrify() done")
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        print(1)
        
def sql_insert_execution(cur, df, table_name, clause='ON CONFLICT DO NOTHING'):
    """
    sdfa
    """
    sql = f"""
    CREATE TABLE {table_name}_h AS
    TABLE {table_name};
    
    COPY {table_name}_h FROM STDIN With CSV;

    INSERT INTO {table_name}
    SELECT *
    FROM {table_name}_h {clause};

    DROP TABLE {table_name}_h;
    """
    file = './{}.csv'.format(table_name)
    ix = False
    if table_name == 'songplays':
        ix = True
    if type(df) != type(pd.DataFrame()):
        with open(file,'w') as out:
            csv_out=csv.writer(out)
            csv_out.writerows(df)
    else:
        df.to_csv(file, index=ix, header=False)
    with open(file, 'r') as f:
        cur.copy_expert(sql,f)
    os.remove(file)

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
%%time
iter_file = iter_song_data('data/song_data/')
song_data, artist_data = song_data_extract(iter_file)

In [ ]:
%%time
#insert_mogrify(cur, song_data, song_table_insert, song_values, song_clause)
#insert_mogrify(cur, artist_data, artist_table_insert, artist_values, artist_clause)

In [ ]:
%%time
sql_insert_execution(cur, song_data, 'songs', clause=song_clause)
sql_insert_execution(cur, artist_data, 'artists', clause=artist_clause)

In [ ]:
%%time
log_file = log_file_extract('data/log_data')
time_df,user_df,songplay_data = log_data_extract(cur,log_file)

In [ ]:
%%time
sql_insert_execution(cur, time_df,'time', time_clause)
sql_insert_execution(cur, user_df,'users', user_clause)
sql_insert_execution(cur, songplay_data,'songplays', songplay_clause)